`launch.json` 文件夹的路径配置
```json
${workspaceRoot} // 当前打开的文件夹的绝对路径

${workspaceRootFolderName} // 当前打开的文件夹的名字

${file} // 当前打开正在编辑的文件名，包括绝对路径，文件名，文件后缀名

${relativeFile} // 从当前打开的文件夹到当前打开的文件的路径

${fileBasename} // 当前打开的文件名+后缀名，不包括路径

${fileBasenameNoExtension} // 当前打开的文件的文件名，不包括路径和后缀名

${fileDirname} // 当前打开的文件所在的绝对路径，不包括文件名

${fileExtname} // 当前打开的文件的后缀名
```

`data.py`
